In [38]:
import os
import pickle

import numpy as np
import pandas as pd
from tqdm import tqdm
import json
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import cv2

In [39]:
image_size = (512, 512)


In [42]:
mhd_info_csv_path = "data/mhd_info.csv"
mhd_info_df = pd.read_csv(mhd_info_csv_path)
mhd_info_df.head()

,mhd_file_path,mask_file_path,subset_id,series_id,total_images,origins,spacings,is_flip,ct_image_folder
0,../downloads/data/subset0/1.3.6.1.4.1.14519.5....,../downloads/seg-lungs-LUNA16/1.3.6.1.4.1.1451...,0,1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031...,121,"[-335.209991, -195.0, -198.100006]","[2.5, 0.7617189884185791, 0.7617189884185791]",False,../data/images/1.3.6.1.4.1.14519.5.2.1.6279.60...
1,../downloads/data/subset0/1.3.6.1.4.1.14519.5....,../downloads/seg-lungs-LUNA16/1.3.6.1.4.1.1451...,0,1.3.6.1.4.1.14519.5.2.1.6279.6001.108197895896...,119,"[-313.75, -190.0, -182.5]","[2.5, 0.7421879768371582, 0.7421879768371582]",False,../data/images/1.3.6.1.4.1.14519.5.2.1.6279.60...
2,../downloads/data/subset0/1.3.6.1.4.1.14519.5....,../downloads/seg-lungs-LUNA16/1.3.6.1.4.1.1451...,0,1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524...,161,"[-194.0, -108.300003, -187.699997]","[1.25, 0.5488280057907104, 0.5488280057907104]",False,../data/images/1.3.6.1.4.1.14519.5.2.1.6279.60...
3,../downloads/data/subset0/1.3.6.1.4.1.14519.5....,../downloads/seg-lungs-LUNA16/1.3.6.1.4.1.1451...,0,1.3.6.1.4.1.14519.5.2.1.6279.6001.111172165674...,538,"[-298.774994, -143.100006, -180.600006]","[0.625, 0.7421879768371582, 0.7421879768371582]",False,../data/images/1.3.6.1.4.1.14519.5.2.1.6279.60...
4,../downloads/data/subset0/1.3.6.1.4.1.14519.5....,../downloads/seg-lungs-LUNA16/1.3.6.1.4.1.1451...,0,1.3.6.1.4.1.14519.5.2.1.6279.6001.122763913896...,124,"[-337.480011, -157.5, -166.399994]","[2.5, 0.6152340173721313, 0.6152340173721313]",False,../data/images/1.3.6.1.4.1.14519.5.2.1.6279.60...


In [43]:
annotations_path = "../downloads/annotations.csv"
annotations_df = pd.read_csv(annotations_path, delimiter=",")
annotations_df.head()

,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.783651,-211.925149,-227.121250,4.224708
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,69.639017,-140.944586,876.374496,5.786348
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,-24.013824,192.102405,-391.081276,8.143262
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,2.441547,172.464881,-405.493732,18.545150


In [44]:
def read_detection_result():
    detection_result_folder = "../output/valresults/caltech/h/off"

    detection_dfs = {}
    for epoch_id in tqdm(os.listdir(detection_result_folder), desc="test epoch"):
        current_folder_path = "{}/{}".format(detection_result_folder, epoch_id)

        detection_dfs.setdefault(epoch_id, {})
        for filename in os.listdir(current_folder_path):
            series_id = filename[0:-4]
            file = os.path.join(current_folder_path, filename)
            result_df = pd.read_csv(file, delimiter=" ", header=None)
            result_df.columns = ["z-index", "top_left_x", "top_left_y", "width", "height", "possibility"]
            detection_dfs.get(epoch_id).setdefault(series_id, result_df)

    return detection_dfs

In [8]:
test_detections = read_detection_result()

test epoch: 100%|██████████| 58/58 [00:07<00:00,  7.91it/s]


In [59]:
for epoch_id in tqdm(test_detections.keys(), desc="epoch"):
    epoch_test_result = test_detections[epoch_id]

    hit_counter = 0

    for series_id in epoch_test_result.keys():
        mhd_info = mhd_info_df[mhd_info_df["series_id"] == series_id].iloc[0]
        origins = mhd_info["origins"]
        spacings = mhd_info["spacings"]
        is_flip = mhd_info["is_flip"]

        spacings = json.loads(spacings)
        origins = json.loads(origins)

        [z_spacing, y_spacing, x_spacing] = spacings
        [z_origin, y_origin, x_origin] = origins

        image_detections = epoch_test_result[series_id]
        # image_detections["z-index"] = image_detections["z-index"]

        total_number_of_detections = len(image_detections)

        nodules = annotations_df[annotations_df["seriesuid"] == series_id]

        for _, nodule in nodules.iterrows():
            print(nodule)
            x = nodule["coordX"]
            y = nodule["coordY"]
            z = nodule["coordZ"]
            diameter = nodule["diameter_mm"]

            if diameter < 0:
                diameter = 10

            radius_squared = pow((diameter / 2), 2)

            matched_detections = []

            for _, detection in image_detections.iterrows():
                x2 = detection["top_left_x"]
                y2 = detection["top_left_y"]
                z2 = detection["z-index"]
                w = detection["width"]
                h = detection["height"]
                p = detection["possibility"]

                x2 = x2 * x_spacing + x_origin
                y2 = y2 * y_spacing + y_origin
                z2 = z2 * z_spacing + z_origin

                print(x2, y2, z2)
                # dist = pow(x - x2, 2) + pow(y - y2, 2) + pow(z - z2, 2)
                # if dist < radius_squared:
                    # matched_detections.append(detection)

                break
            break



        break
    break

epoch:   0%|          | 0/58 [00:00<?, ?it/s]

seriesuid      1.3.6.1.4.1.14519.5.2.1.6279.6001.102681962408...
coordX                                                105.055792
coordY                                                  19.82526
coordZ                                                -91.247251
diameter_mm                                            21.089619
Name: 8, dtype: object
-39.546349453125 89.032304453125 -92.0


In [ ]:
def evaluation():
# dist = math.pow(x - x2, 2.) + math.pow(y - y2, 2.) + math.pow(z - z2, 2.)
# if dist < radiusSquared: